In [ ]:
MAKE A CONNECTION WITH DATABASE

In [17]:
import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="s@qiekausarali5",
  database="biva"
)
mycursor = mydb.cursor()
print ("Hello World")

Hello World


In [ ]:
DATA EXPLORATION

In [27]:
#1
mycursor.execute("select c.category, c.subcategory, count(product_id) as product_count from category_dim c inner join product_dim p on c.category_id = p.category_id group by c.category, c.subcategory;")
df = pd.DataFrame(mycursor.fetchall())
df = df.rename(columns = {0:'category', 1:'subcategory', 2:'products'})
print (df)

           category  subcategory  products
0         Furniture    Bookcases       583
1         Furniture       Chairs       647
2         Furniture  Furnishings       719
3         Furniture       Tables       381
4   Office Supplies   Appliances       587
5   Office Supplies          Art       708
6   Office Supplies      Binders       781
7   Office Supplies    Envelopes       613
8   Office Supplies    Fasteners       583
9   Office Supplies       Labels       602
10  Office Supplies        Paper       825
11  Office Supplies      Storage       683
12  Office Supplies     Supplies       573
13       Technology  Accessories       700
14       Technology      Copiers       537
15       Technology     Machines       514
16       Technology       Phones       732


In [21]:
#2
mycursor.execute("select p.product_name, c.category, c.subcategory, p.unit_price from category_dim c inner join product_dim p on c.category_id = p.category_id;")
df = pd.DataFrame(mycursor.fetchall())
df = df.rename(columns = {0:'product', 1:'category', 2:'subcategory', 3:'price'})
print (df)

                                        product    category subcategory  \
0                 Bush Classic Bookcase, Mobile   Furniture   Bookcases   
1        Sauder Library with Doors, Traditional   Furniture   Bookcases   
2                  Dania Corner Shelving, Metal   Furniture   Bookcases   
3         Dania Floating Shelf Set, Traditional   Furniture   Bookcases   
4                  Safco Classic Bookcase, Pine   Furniture   Bookcases   
...                                         ...         ...         ...   
10763             Samsung Smart Phone, Cordless  Technology      Phones   
10764            Samsung Office Telephone, VoIP  Technology      Phones   
10765       Samsung Smart Phone, with Caller ID  Technology      Phones   
10766  Samsung Office Telephone, with Caller ID  Technology      Phones   
10767            Samsung Smart Phone, Full Size  Technology      Phones   

          price  
0      415.2000  
1      232.5780  
2      121.5300  
3      152.4150  
4      21

PIE CHARTS

In [220]:
from math import pi
import pandas as pd
from bokeh.io import output_notebook, output_file, show
from bokeh.palettes import Category20c, Spectral10
from bokeh.plotting import figure
from bokeh.transform import cumsum, factor_cmap
from bokeh.transform import dodge
from bokeh.models import ColumnDataSource
output_file("plot.html")

In [29]:
#5.	Pie chart to show all regions and measuring country instances in them
mycursor.execute("select region, count(*) from (select distinct region, country from location_dim) a group by a.region;")
df = pd.DataFrame(mycursor.fetchall())
df = df.rename(columns = {0:'region', 1:'count'})
print (df)

            region  count
0           Africa     45
1     Central Asia      6
2          Oceania      3
3   Southeast Asia      8
4       North Asia      6
5           Canada      1
6             EMEA     40
7          Central     12
8            North      7
9            South     14
10       Caribbean      8
11            West      1
12            East      1


In [49]:
df['angle'] = df['count']/df['count'].sum() *2* pi
df['color'] = Category20c[len(df)]
df

,region,count,angle,color
0,Africa,45,1.860154,#3182bd
1,Central Asia,6,0.248020,#6baed6
2,Oceania,3,0.124010,#9ecae1
3,Southeast Asia,8,0.330694,#c6dbef
4,North Asia,6,0.248020,#e6550d
5,Canada,1,0.041337,#fd8d3c
6,EMEA,40,1.653470,#fdae6b
7,Central,12,0.496041,#fdd0a2
8,North,7,0.289357,#31a354
9,South,14,0.578714,#74c476


In [53]:
p = figure(plot_height=500, 
            title="Country Count in Regions",
            tools="hover", 
            tooltips=[("Region", "@region"), ("Countries", "@count")], 
            x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, 
        radius=0.5, 
        start_angle=cumsum('angle', include_zero=True), 
        end_angle=cumsum('angle'),
        line_color="white", 
        fill_color='color', 
        legend_field='region', 
        source=df)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None


In [54]:
show(p)

BAR CHART & HISTOGRAM

In [175]:
#6.	histogram to show the states in a country on x-axis and the count of cities in each state on y-axis
mycursor.execute("select state, count(city) from (select distinct country, state, city from location_dim where country = 'China') a group by a.state;")
df = pd.DataFrame(mycursor.fetchall())
df = df.rename(columns = {0:'State', 1:'count'})
print(df)

             State  count
0            Anhui     11
1          Beijing      3
2        Chongqing      3
3           Fujian      7
4            Gansu      2
5        Guangdong     22
6          Guangxi      6
7          Guizhou      4
8           Hainan      2
9            Hebei     10
10    Heilongjiang     16
11           Henan     15
12           Hubei     19
13           Hunan     14
14  Inner Mongolia      6
15         Jiangsu     19
16         Jiangxi      4
17           Jilin     12
18        Liaoning     12
19         Ningxia      1
20         Qinghai      1
21         Shaanxi      4
22        Shandong     20
23        Shanghai      1
24          Shanxi      8
25         Sichuan     12
26         Tianjin      3
27  Xinjiang Uygur      3
28          Yunnan      2
29        Zhejiang     14


In [179]:
p = figure(x_range = df['State'], 
            title="Cities in States of China", 
            y_axis_label = "No. of Cities", 
            tools="pan,box_select,zoom_in,zoom_out,save,reset,hover", 
            tooltips=[("State", "@State"), ("cities", "@count")])

p.vbar(x = 'State', 
        top = 'count', 
        width = 1.0, 
        source = df, 
        line_color="white", 
        color = '#3182bd')
p.xaxis.axis_label = "States in China"
p.xaxis.major_label_orientation = 0.9
p.x_range.range_padding = 0.05
show(p)

In [158]:
#33. Total lifetime sales of all subcategories in all categories (Segmented Bar chart)
mycursor.execute("select c.category, c.subcategory, sum(sales) sales from category_dim c inner join mv_product_sales m on c.category_id = m.category_id group by c.category, c.subcategory;")
df = pd.DataFrame(mycursor.fetchall())
df = df.rename(columns = {0:'category', 1:'subcategory', 2: 'sales'})
print(df)

           category  subcategory         sales
0         Furniture  Furnishings   385291.8759
1         Furniture       Tables   757041.9244
2         Furniture    Bookcases  1466404.2118
3         Furniture       Chairs  1500296.4972
4   Office Supplies      Binders   461504.4627
5   Office Supplies    Fasteners    83242.3159
6   Office Supplies     Supplies   243074.2206
7   Office Supplies    Envelopes   170883.8116
8   Office Supplies   Appliances  1007347.9530
9   Office Supplies          Art   371560.1349
10  Office Supplies       Labels    73392.6600
11  Office Supplies        Paper   244196.9194
12  Office Supplies      Storage  1126425.7414
13       Technology  Accessories   748785.9625
14       Technology       Phones  1703104.8592
15       Technology      Copiers  1509436.2742
16       Technology     Machines   778553.4271


In [159]:
group = df.groupby(by=['category', 'subcategory'])

index_cmap = factor_cmap('category_subcategory', palette=Spectral10, factors=sorted(df.category.unique()), end=1)

p = figure(plot_width=700, 
            plot_height=500, 
            title="Total Sales of Subcategories", 
            x_range=group, 
            tools="pan,box_select,zoom_in,zoom_out,save,reset,hover", 
            tooltips=[("Category_SubCategory: ", "@category_subcategory"), ("sales: ", "@sales_top")])

p.vbar(x='category_subcategory', 
        top='sales_top', width=1, 
        source=group, 
        line_color="white", 
        fill_color=index_cmap)

p.y_range.start = 0
p.x_range.range_padding = 0.05
p.xgrid.grid_line_color = None
p.xaxis.axis_label = "Subcategories in Categories"
p.yaxis.axis_label = "Sales (in Million USD)"
p.xaxis.major_label_orientation = 1.0
p.outline_line_color = 'black'
show(p)

In [195]:
#18. Total sales by markets this month vs previous month (in grouped bar chart)
mycursor.execute("select a.market, a.sales this_month, b.sales prev_month from (select market, sum(sales) sales from mv_location_time_sales where year=2014 and month=12 group by market) a inner join (select market, sum(sales) sales from mv_location_time_sales where year=2014 and month=11 group by market) b on a.market = b.market;")
df = pd.DataFrame(mycursor.fetchall())
df = df.rename(columns = {0:'market', 1:'this_month', 2: 'prev_month'})
print(df)

   market   this_month   prev_month
0  Africa   33802.7310   37862.8590
1    APAC  121584.0093  155630.0382
2  Canada    5784.1200    2495.1900
3    EMEA   40010.4690   21275.4690
4      EU  123845.6595  119018.4330
5   LATAM   87497.2789  106502.5367
6      US   90372.2648  111782.0910


In [196]:
p = figure(x_range = df['market'], 
            title="Current and Previous month sales by market", 
            y_axis_label = "Sales Amount", 
            tools="pan,box_select,zoom_in,zoom_out,save,reset,hover", 
            tooltips=[("Market", "@market"), ("Nov 2014", "@this_month"), ("Nov 2014", "@prev_month")])

p.vbar(x=dodge('market', -0.15, range=p.x_range), 
        top = 'this_month', 
        width = 0.3, 
        source = df, 
        line_color="white", 
        color = '#3182bd', 
        fill_alpha = 0.8,
        legend_label='Dec 2014')
p.vbar(x=dodge('market', +0.15, range=p.x_range), 
        top = 'prev_month', 
        width = 0.3, 
        source = df, 
        line_color="white", 
        color = '#e6550d', 
        fill_alpha = 0.8,
        legend_label='Nov 2014')
p.xaxis.axis_label = "Sales in USD"
#p.xaxis.major_label_orientation = 0.9
p.x_range.range_padding = 0.05
p.y_range.start = 0
show(p)

In [197]:
#38. Stacked bar cart to show sales and discount of category X in every market
# taking category X as Furniture
mycursor.execute("select l.market, sum(s.sales) sales, sum(s.discount) discount from (select * from category_dim where    category = 'Furniture') c inner join product_dim p on c.category_id = p.category_id inner join sales_fact s on p.product_id = s.product_id inner join location_dim l on s.location_id = l.location_id group by l.market;")
df = pd.DataFrame(mycursor.fetchall())
df = df.rename(columns = {0:'market', 1:'sales', 2: 'discount'})
print(df)

   market         sales discount
0      EU   778958.2245   177101
1    APAC  1343422.2105   248899
2   LATAM   811355.3800   215314
3      US   741432.0433   185113
4    EMEA   228620.7360    64926
5  Africa   194650.6350    45309
6  Canada    10595.2800        0


In [198]:
p = figure(x_range = df['market'], 
            title="Sales and Discount of X in all Markets", 
            y_axis_label = "Amount in USD", 
            tools="pan,box_select,zoom_in,zoom_out,save,reset,hover", 
            tooltips="$name @market: @$name")
colors = ["#3182bd", "#e6550d"]
measures = ['sales','discount']
p.vbar_stack(measures, 
            x='market', 
            width=0.9, 
            color=colors, 
            source=df,
            fill_alpha = 0.8,
            legend_label=measures)
p.xaxis.axis_label = "Sales in USD"
#p.xaxis.major_label_orientation = 0.9
p.x_range.range_padding = 0.05
p.y_range.start = 0
show(p)

LINE PLOTS

In [199]:
#25. Live comparison of this year’s weekly cumulative sales vs previous year’s weekly cumulative sales.
mycursor.execute("select a.week, sum(a.sales) over(order by week) as S2014, sum(b.sales) over(order by week) as S2013 from (select week, sum(sales) sales from mv_location_time_sales where year = 2014 group by week) a inner join (select week, sum(sales) sales from mv_location_time_sales where year = 2013 group by week) b on a.week = b.week;")
df = pd.DataFrame(mycursor.fetchall())
df = df.rename(columns = {0:'week', 1:'2014', 2: '2013'})
print(df)

    week          2014          2013
0      0    23816.4463    48836.5905
1      1    73070.5054    92706.1145
2      2   137252.0760   125254.7939
3      3   193912.4421   161671.6243
4      4   242135.0516   213748.8916
5      5   281581.3840   266257.7387
6      6   340903.2180   299349.5742
7      7   386085.4578   337816.2597
8      8   428945.7489   380288.6539
9      9   468148.7199   427889.4183
10    10   525391.0285   484835.4342
11    11   597338.5698   525968.5578
12    12   674976.0828   564585.8401
13    13   729935.1510   602031.1712
14    14   788332.6939   643992.2318
15    15   851677.9915   691133.2145
16    16   903554.6012   727964.4753
17    17   955032.2687   774375.8364
18    18  1028785.8559   823724.0141
19    19  1090744.6062   872827.8694
20    20  1153467.0319   929321.8355
21    21  1220306.1062  1011267.5945
22    22  1303175.4439  1102588.3215
23    23  1393075.0639  1214073.9326
24    24  1495394.8293  1292327.6516
25    25  1600186.3120  1394788.5155
2

In [236]:
sc = ColumnDataSource(df)
p = figure(plot_width=800, plot_height=400, title="Weekly Sales Comparison", 
            y_axis_label = "Sales in USD", 
            tools="pan,box_select,zoom_in,zoom_out,save,reset,hover", 
            tooltips=[("Week: ", "@week"), ("2014: ", "@2014"), ("2013: ", "@2013")])

# add a line renderer
p.line('week', '2014', source = df, line_width=3, color = '#3182bd', legend_label='2014')
p.line('week', '2013', source = df, line_width=3, color = '#e6550d', legend_label='2013') 
p.legend.location = "top_left"
show(p)

In [238]:
import networkx as nx

from bokeh.io import output_file, show
from bokeh.models import (BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool,
                          MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool,)
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx

G=nx.karate_club_graph()

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
plot.title.text = "Graph Interaction Demonstration"

plot.add_tools(HoverTool(tooltips=None), TapTool(), BoxSelectTool())

graph_renderer = from_networkx(G, nx.circular_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=5)

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(graph_renderer)

output_file("interactive_graphs.html")
show(plot)